# Lab 3 · Estimating Elimination Rate from Plasma Samples

We will derive the elimination rate constant (k) for an IV bolus study by linearizing the concentration–time curve.

### Learning goals
- Plot concentration vs. time to understand the dosing profile.
- Use the natural log transform to make exponential decay linear.
- Fit linear regression on `time_hr` vs `ln(concentration)`.
- Convert the slope into the elimination rate constant and half-life.

### Workflow
1. Load `../data/pk_concentration_time.csv` with pandas.
2. Visualize the raw data to confirm an exponential decrease.
3. Create a `ln_concentration` column.
4. Fit a linear regression to estimate the decay slope.
5. Report the elimination constant and half-life.

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

data_path = Path('..') / 'data' / 'pk_concentration_time.csv'
df = pd.read_csv(data_path)
df

### Exercise 1 · Plot the concentration–time curve

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(df['time_hr'], df['concentration_mg_per_L'], marker='o')
plt.xlabel('Time (hr)')
plt.ylabel('Concentration (mg/L)')
plt.title('Single-dose concentration profile')
plt.show()

### Exercise 2 · Log-transform the concentrations
Taking the natural log of concentration lets us model the curve with a straight line.

In [ ]:
df['ln_concentration'] = np.log(df['concentration_mg_per_L'])
df[['time_hr', 'concentration_mg_per_L', 'ln_concentration']].head()

### Exercise 3 · Fit linear regression to ln(concentration)

In [ ]:
X = df[['time_hr']]
y = df['ln_concentration']

model = LinearRegression()
model.fit(X, y)

slope = model.coef_[0]
intercept = model.intercept_
print(f"Slope (k): {slope:.4f}")
print(f"Intercept: {intercept:.4f}")

pred_ln = model.predict(X)
r2 = r2_score(y, pred_ln)
print(f"R² on log-scale: {r2:.3f}")

df['pred_ln_concentration'] = pred_ln
df['pred_concentration'] = np.exp(pred_ln)
df[['time_hr', 'concentration_mg_per_L', 'pred_concentration']]

### Exercise 4 · Estimate the elimination half-life
Remember: `half_life = ln(2) / k`. The slope is negative, so we flip the sign.

In [ ]:
elimination_rate_constant = -slope
half_life = np.log(2) / elimination_rate_constant
print(f"Elimination rate constant k: {elimination_rate_constant:.4f} 1/hr")
print(f"Estimated half-life: {half_life:.2f} hr")

### Try it yourself
- Remove one of the later time points and refit the model. How does half-life change?
- What happens if you use only the first 6 hours of data?